 PyTorch/ResNet18 기반 이미지 검색을 TensorFlow/Keras 기반의 ResNet50으로 구현한 예시

In [ ]:
!pip install chromadb sentence-transformers torchvision

In [ ]:
import numpy as np
import tensorflow as tf
import os
from PIL import Image # 이미지 파일 처리를 위한 라이브러리
from chromadb import PersistentClient # ChromaDB 클라이언트

# 1. ResNet50 모델 로드 및 설정 (특징 추출기 역할)
model = tf.keras.applications.ResNet50(
    include_top = False,  # 핵심: 최종 분류층(Fully Connected Layer)을 제거
    pooling='avg',        # 특징 맵을 평균 풀링(Global Average Pooling)하여 고정된 길이의 벡터로 변환
    weights = 'imagenet', # ImageNet으로 사전 학습된 가중치 사용 (전이 학습)
    input_shape=(224, 224, 3) # 입력 이미지 크기 정의 (TensorFlow는 기본적으로 H, W, C 순서)
)
# ResNet50의 pooling='avg' 설정은 2048차원의 특징 벡터를 출력함 (ResNet18은 512차원이었음)

# 2. 이미지를 불러와 벡터로 변환하는 함수 (TensorFlow 버전)
def image_to_vector_tf(img_path):
  image = Image.open(img_path).convert('RGB').resize((224,224)) # 이미지 로드 및 크기 조정 (224x224)
  img_array = np.array(image) # PIL 이미지를 Numpy 배열로 변환
  img_array = tf.keras.applications.resnet50.preprocess_input(img_array)   
  # 핵심: ResNet50이 요구하는 전처리 수행 (픽셀 값의 스케일 조정 및 평균/표준편차 보정)
  img_array = np.expand_dims(img_array, axis=0)
  # 모델에 넣기 위해 배치 차원(앞에 1) 추가. shape: (224, 224, 3) -> (1, 224, 224, 3)
  # 모델 예측 실행 (추론)
  vector = model.predict(img_array)[0]  # 결과는 (1, 2048) -> [0]을 통해 (2048,) 벡터만 추출
  print(vector)
  return vector.tolist()

In [ ]:
# 3. ChromaDB 설정 및 데이터 준비
client = PersistentClient(path = '.\image_chroma_tf') # DB 경로 설정
collection = client.get_or_create_collection(name='image_tf')
image_files = ['apple.jpeg', 'grape.jpeg', 'peach.jpeg']
ids = [f'img{i}' for i in range(len(image_files))]
print(ids)

# 4. 이미지 벡터화 및 DB 저장 (Indexing)
for img_id, img_path in zip(ids, image_files):
  if not os.path.exists(img_path):
    print(f'파일없음:{img_path}')
    continue
  vec = image_to_vector_tf(img_path)
  collection.add(
      embeddings=[vec],
      documents=[img_path],
      ids=[img_id],
      metadatas = [{'filename':img_path}]
  )

<>:2: SyntaxWarning: invalid escape sequence '\i'
<>:2: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipython-input-1942468124.py:2: SyntaxWarning: invalid escape sequence '\i'
  client = PersistentClient(path = '.\image_chroma_tf')


['img0', 'img1', 'img2']
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step
[2.8431735  0.27945992 0.11864086 ... 2.0457733  0.03444489 0.00733102]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
[0.43031392 0.00671981 1.8788161  ... 0.02374941 0.10511564 0.        ]
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
[0.93436486 0.00664525 0.03079196 ... 0.17640269 0.12955387 0.        ]


In [ ]:
from chromadb.api.types import Document
# 5. 쿼리 이미지 검색 (Retrieval)
query_path = 'apple.jpeg'
if not os.path.exists(query_path):
  print(f'쿼리이미지 없음 : {query_path}')
else:
  query_vec = image_to_vector_tf(query_path)

  # ChromaDB 클라이언트 재연결 및 컬렉션 가져오기 (컬렉션 상태 보장)
  client = PersistentClient(path = '.\image_chroma_tf')
  collection = client.get_collection(name='image_tf')

  # 유사도 검색 실행
  results = collection.query(
      query_embeddings=[query_vec],
      n_results=3,
      include = ['documents', 'distances',  'metadatas']
  )
  print('검색 결과 :')
  # 검색 결과 출력 (메타데이터 존재 여부 확인 로직 포함)
  # Check if the first element of metadatas is not None before iterating
  if results and results.get('metadatas') and results['metadatas'] and results['metadatas'][0] is not None:
    for doc, dist, meta in zip(results['documents'][0], results['distances'][0], results['metadatas'][0]):
      # Add a check for meta being None inside the loop just in case
      if meta is not None:
        print(f'{meta['filename']} (유사도 거리 : {dist:.4f})')
      else:
        print("Skipping a result with no metadata.")
  else:
    print("No similar images found.")

<>:10: SyntaxWarning: invalid escape sequence '\i'
<>:10: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipython-input-3068283381.py:10: SyntaxWarning: invalid escape sequence '\i'
  client = PersistentClient(path = '.\image_chroma_tf')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
[2.8431735  0.27945992 0.11864086 ... 2.0457733  0.03444489 0.00733102]
검색 결과 :
Skipping a result with no metadata.
Skipping a result with no metadata.
Skipping a result with no metadata.


추가 개념 정리: TensorFlow와 PyTorch의 이미지 벡터화 비교


이미지 벡터화를 PyTorch와 TensorFlow 두 가지 프레임워크로 모두 구현. 이 둘의 주요 차이점은 전처리 방식과 모델 설정.

1. Keras/TensorFlow 전처리: preprocess_input의 중요성
PyTorch 실습에서는 transforms.ToTensor()가 기본적인 0\~1 정규화를 담당. 하지만 Keras의 사전 학습된 모델은 더 복잡한 전처리를 요구.

tf.keras.applications.resnet50.preprocess_input(img_array):
이 함수는 ImageNet 데이터셋으로 ResNet50을 훈련할 때 사용된 방식과 동일하게 픽셀 값을 조정.
주로 픽셀 값을 특정 범위로 스케일링하거나, 채널별 평균과 표준편차를 빼주는 작업이 포함.

핵심: 이 전처리 과정을 거치지 않으면, 벡터는 제대로 된 특징을 추출하지 못하고 검색 정확도가 크게 떨어짐. 사전 학습된 모델을 사용할 때는 반드시 해당 모델의 전처리 함수.


2. 벡터 차원의 차이: ResNet18 vs. ResNet50

모델	        사용 프레임워크	설정	            출력 벡터 차원
ResNet18	PyTorch (model.children()[:-1])	    최종 풀링 계층 전 특징 사용	512차원
ResNet50	TensorFlow (pooling='avg')	        최종 분류층 제거 및 평균 풀링 적용	2048차원


pooling='avg': Keras의 applications 모델에서 include_top=False와 함께 pooling='avg'를 사용하면, 컨볼루션 레이어 마지막에 Global Average Pooling을 적용하여 이미지 특징을 고정된 길이(2048)의 벡터로 압축.
(컨볼루션 레이어는 이미지의 특징을 추출하여 위치를 보존함으로써, 컴퓨터가 데이터를 더 정확하게 인식하고 복잡한 시각적 개념을 효율적으로 이해하도록 돕는 핵심 메커니즘)
차원 크기: ResNet50은 ResNet18보다 더 깊은 모델이므로 더 많은 특징 정보를 추출할 수 있으며, 이 때문에 최종 임베딩 벡터의 차원이 2048로 훨씬 커짐. (일반적으로 차원이 클수록 더 풍부한 정보를 담을 가능성이 높지만, 벡터 DB의 저장 공간과 검색 속도에는 영향 줌.)


3. 코드 안정성 (실무적 관점)
코드를 보시면 if results and results.get('metadatas') and results['metadatas'] and results['metadatas'][0] is not None:와 같이 검색 결과가 없을 경우나 메타데이터가 없는 경우를 대비하는 안전 코드를 추가. 
이는 실무에서 시스템이 오류 없이 안정적으로 동작하도록 보장하는 방어적 프로그래밍의 좋은 예